# **Implementation of pre-trained model for TTS**
Please refer to the respective sections in the book for further details.


## **Step 1. Installing Libraries and Data loading**

In [ ]:
!pip install huggingface-hub
!pip install datasets
!pip install transformers datasets soundfile speechbrain accelerate

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset, Audio, DatasetDict, Audio
from collections import defaultdict
import os
import torch
from speechbrain.pretrained import EncoderClassifier

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
welsh_voice_train_dataset = load_dataset("mozilla-foundation/common_voice_13_0", "cy", split="train")
welsh_voice_train_dataset = welsh_voice_train_dataset.cast_column("audio", Audio(sampling_rate=16000))

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 7810it [00:00, 137681.74it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 5293it [00:00, 129379.29it/s]


...


Reading metadata...: 5307it [00:00, 132586.62it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 18680it [00:00, 134067.53it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 4390it [00:00, 130308.59it/s]


## **Step 2. Data pre-processing**

In [ ]:
client_id_to_filter = 'ccdf75e8e5f1a3c84918a8616f520e605825c284dc124dfadeb3f1d36e26f4a86a893075dc03589ff4996b449f273627a9fb0c39f2b76a20227567faee343016'
welsh_voice_train_filtered_dataset = welsh_voice_train_dataset.filter(lambda example: example['client_id'] == client_id_to_filter)

Filter:   0%|          | 0/7810 [00:00<?, ? examples/s]

In [ ]:
speaker_recognition_model_name = "speechbrain/spkrec-xvect-voxceleb"
computation_device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_recognition_model = EncoderClassifier.from_hparams(
    source=speaker_recognition_model_name,
    run_opts={"device": computation_device, "timeout": 30},
    savedir=os.path.join("/tmp", speaker_recognition_model_name),
)

def generate_speaker_embedding(audio_waveform):
    with torch.no_grad():
        embeddings = speaker_recognition_model.encode_batch(torch.tensor(audio_waveform))
        normalized_embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        flattened_embeddings = normalized_embeddings.squeeze().cpu().numpy()
    return flattened_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
def process_audio_data(sample):
    audio_data = sample["audio"]
    processed_sample = tts_processor(
        text=sample["sentence"],
        audio_target=audio_data["array"],
        sampling_rate=audio_data["sampling_rate"],
        return_attention_mask=False,
    )

    processed_sample["labels"] = processed_sample["labels"][0]
    processed_sample["speaker_embeddings"] = generate_speaker_embedding(audio_data["array"])
    return processed_sample

In [ ]:
tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
welsh_voice_train_filtered_dataset = welsh_voice_train_filtered_dataset.map(process_audio_data, remove_columns=welsh_voice_train_filtered_dataset.column_names)

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
sample_data = welsh_voice_train_filtered_dataset[1]
sample_speaker_embeddings = torch.tensor(sample_data["speaker_embeddings"]).unsqueeze(0)

## **Step 3. Model Inference**

In [ ]:
tts_input_parameters = tts_processor(text="Doedd hi ddim wedi arfer gyda'r math yma o beth chwaith.", return_tensors="pt")
speech_synthesis_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [ ]:
audio_generator = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
generated_audio = speech_synthesis_model.generate_speech(tts_input_parameters["input_ids"], sample_speaker_embeddings, vocoder=audio_generator)

In [ ]:
from IPython.display import Audio
Audio(generated_audio, rate=16000)